In [1]:
import sys
sys.path[0]+='/..'
# from graph_dataset.dataset import GraphDataset
from rgcn_dataset import GraphDataset

from utilities import *

import torch
import torch_geometric
import os 
import time
from torch.optim.lr_scheduler import CosineAnnealingLR
from uc_class import UC

import gurobipy as gp
import json
from torch_geometric.data import HeteroData
import collections


In [2]:
import copy

class BipartiteGraphConvolution(torch_geometric.nn.MessagePassing):
    """
    The bipartite graph convolution is already provided by pytorch geometric and we merely need
    to provide the exact form of the messages being passed.
    """

    def __init__(self,emb_size):
        super().__init__("add")
            
        self.feature_module_left = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size)
        )
        self.feature_module_edge = torch.nn.Sequential(
            torch.nn.Linear(2, emb_size, bias=False)
        )
        self.feature_module_right = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size, bias=False)
        )
        self.feature_module_final = torch.nn.Sequential(
            torch.nn.LayerNorm(emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )
        
    

    def forward(self, left_features, edge_indices, edge_features, right_features):
        output = self.propagate(
            edge_indices,
            size=(left_features.shape[0], right_features.shape[0]),
            node_features=(left_features, right_features),
            edge_features=edge_features,
        )
        return output
            
    def message(self, node_features_i, node_features_j, edge_features):
        output = self.feature_module_final(
            self.feature_module_left(node_features_i)
            + self.feature_module_edge(edge_features)
            + self.feature_module_right(node_features_j)
        )
        return output
    


class GNNPolicy(torch.nn.Module):
    def __init__(self):
        super(GNNPolicy, self).__init__()
        emb_size=64
        self.var_node = ['u','p','s']
        self.con_node = ['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
        self.obj_node = ['obj']
        self.node_types=['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost','obj']
        self.relations = ['v2o','o2c','v2c','c2o','o2v','c2v']
        self.target_nodes = [self.obj_node,self.con_node,self.con_node,self.obj_node,self.var_node,self.var_node]
        self.source_nodes = [self.var_node,self.obj_node,self.var_node,self.con_node,self.obj_node,self.con_node]
        self.init_sizes=[6,6,6,4,4,4,4,4,4,4,1]
        edge_feats=2
        # self.num_relations=len(self.var_node)*len(self.con_node)
        self.vc_idx={}
        self.oc_idx={}
        self.ov_idx={}
        self.node_types = ['u','p','s','Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']    
        self.node_embedding = torch.nn.ModuleList()
        for i in [6,4,1]:
            lin = torch.nn.Sequential(
                torch.nn.LayerNorm(i),
                torch.nn.Linear(i, emb_size),
                torch.nn.ReLU(),
                torch.nn.Linear(emb_size,emb_size),
                torch.nn.ReLU()
            )
            self.node_embedding.append(lin)
        self.edge_embedding = torch.nn.LayerNorm(edge_feats)

        self.con1=torch.nn.Sequential()
        self.emb1=torch.nn.Sequential()
        self.con2=torch.nn.Sequential()
        self.emb2=torch.nn.Sequential()
        for rel in self.relations:
            self.con1.append(BipartiteGraphConvolution(emb_size))
            self.con2.append(BipartiteGraphConvolution(emb_size))
            self.emb1.append(self.down_scale(emb_size))
            self.emb2.append(self.down_scale(emb_size))


        self.ln = torch.nn.LayerNorm(emb_size)


        self.output_module = torch.nn.Sequential(
            torch.nn.Linear(emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, 1, bias=False)
        )
    def down_scale(self,emb_size):
        return torch.nn.Sequential(
            torch.nn.Linear(2 * emb_size, emb_size),
            torch.nn.ReLU(),
            torch.nn.Linear(emb_size, emb_size),
        )

    def trans_dimensions(self, g):
        data = copy.deepcopy(g)
        for v in self.var_node:
            data[v].x = self.node_embedding[0](data[v].x)
        for c in self.con_node:
            data[c].x = self.node_embedding[1](data[c].x)
        data['obj'].x = self.node_embedding[2](data['obj'].x)
        cnt=0
        for v in self.var_node:
            for c in self.con_node:
                if 'edge_attr' in data[v,'v2c',c]:
                    data[v,'v2c',c].edge_attr = self.edge_embedding(data[v,'v2c',c].edge_attr)
                    data[c,'c2v',v].edge_attr = data[v,'v2c',c].edge_attr
                    self.vc_idx[(v,c)]=cnt
                    self.vc_idx[(c,v)]=cnt
                else:
                    self.vc_idx[(v,c)]=-1
                    self.vc_idx[(c,v)]=-1
                cnt+=1
        cnt=0
        for c in self.con_node:
            if 'edge_attr' in data['obj','o2c',c]:
                data['obj','o2c',c].edge_attr = self.edge_embedding(data['obj','o2c',c].edge_attr)
                data[c,'c2o','obj'].edge_attr = data['obj','o2c',c].edge_attr
                self.oc_idx[('obj',c)]=cnt
                self.oc_idx[(c,'obj')]=cnt
            else:
                self.oc_idx[('obj',c)]=-1
                self.oc_idx[(c,'obj')]=-1
            cnt+=1
        cnt=0
        for v in self.var_node:
            if 'edge_attr' in data['obj','o2v',v]:
                data['obj','o2v',v].edge_attr = self.edge_embedding(data['obj','o2v',v].edge_attr)
                data[v,'v2o','obj'].edge_attr = data['obj','o2v',v].edge_attr
                self.ov_idx[('obj',v)]=cnt
                self.ov_idx[(v,'obj')]=cnt
            else:
                self.ov_idx[('obj',v)]=-1
                self.ov_idx[(v,'obj')]=-1
            cnt+=1

        return data
    def process_nodes(self, target_node, source_node, emb, conv, data, edge_type):
        if 'o' in edge_type and 'v' in edge_type:
            idx = self.ov_idx
        elif 'o' in edge_type and 'c' in edge_type:
            idx = self.oc_idx
        else:
            idx = self.vc_idx
        for node in target_node:
            x = torch.zeros_like(data[node].x)
            for other_node in source_node:
                if idx[(other_node, node)] != -1:
                    x += conv(
                        data[other_node].x, 
                        data[other_node, edge_type, node].edge_index, 
                        data[other_node, edge_type, node].edge_attr,
                        data[node].x
                    )
            data[node].x = emb(torch.cat([data[node].x, self.ln(x)], dim=-1))
    
    def forward(self, data):
        data = self.trans_dimensions(data)
        for i,(tar,sou,rel) in enumerate(zip(self.target_nodes,self.source_nodes,self.relations)):
            self.process_nodes(tar, sou, self.emb1[i], self.con1[i], data, rel)
        for i,(tar,sou,rel) in enumerate(zip(self.target_nodes,self.source_nodes,self.relations)):
            self.process_nodes(tar, sou, self.emb2[i], self.con2[i], data, rel)

        x = self.output_module(data['u'].x).sigmoid()
        return x


In [3]:
#set folder
uc_model = '1bin'
instance = '80_c11_based_8_std'
# instance = '80_c11_based_8_std'
# instance = '5_std'
model = 'tripartite_Continuity'

uc_path = f'../UC_AF/{instance}.mod'
uc = UC(uc_path)

shut_down = uc.Pishutdown
startup = uc.Pistartup
on = uc.ThTime_on_min
off = uc.ThTime_off_min
pmax = uc.ThPimax
pmin = uc.ThPimin
ramp_up = uc.Piup
ramp_down = uc.Pidown


model_save_path = f'../model_save/{uc_model}_{instance}_{model}/'
log_save_path =   f"../model_save/{uc_model}_{instance}_{model}_log/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)

log_file = open(f'{log_save_path}train.log', 'wb')

In [4]:
LEARNING_RATE = 1e-4
NB_EPOCHS =1000
BATCH_SIZE = 1
NUM_WORKERS = 2
ETA_MIN=1e-5

In [5]:
train_data = GraphDataset(f'../datasets/{instance}/{uc_model}/{model}/train/')
valid_data = GraphDataset(f'../datasets/{instance}/{uc_model}/{model}/valid/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [6]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
PredictModel = GNNPolicy().to(DEVICE)
if os.path.exists(model_save_path+'model_best.pth'):
    state_dict = torch.load(model_save_path+'model_best.pth')
    PredictModel.load_state_dict(state_dict)
    print("载入模型成功")

载入模型成功


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    acc = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            if optimizer:
                weighted_sol = batch.weighted_sol
            else:
                weighted_sol = batch.opt_sol
            presol = predict(batch).squeeze(1)
            
            loss = loss_fn(presol, weighted_sol)
            preds = (presol > 0.5).float()  # Assuming threshold of 0.5
            acc += torch.sum(preds == batch.opt_sol).item()/len(batch.opt_sol)

            
            if optimizer is not None:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            mean_loss += loss.item()
            n_samples_processed += 1
    mean_loss /= n_samples_processed
    acc /= n_samples_processed
    return mean_loss,acc


In [8]:
def get_norm(ori_tensor):
    ori_tensor = ori_tensor.reshape(-1,1)
    norm = torch.norm(ori_tensor, keepdim=True)
    norm_tensor = (ori_tensor/(norm+1e-6))
    return torch.cat((ori_tensor, norm_tensor), dim=1)


def get_Tripartite_graph(m):
    constraints=['Minimum_up/down_time_constraints','Unit_generation_limits','Power_balance_constrains','System_spinning_reserve_requirement','Ramp_rate_limits','Initial_status_of_units','startup_cost']
    variables=['u','p','s']
    A = m.getA()
    rows, cols = A.shape
    mvars = m.getVars()
    constrs = m.getConstrs()
    b_vars = torch.tensor([1 if 'u'in v.Varname else 0 for v in mvars])
    coo = A.tocoo()
    v0 = torch.tensor(m.getAttr("Obj", mvars)).reshape(-1,1)
    v1 = torch.tensor(coo.getnnz(axis=0)).reshape(-1,1)
    v2 = torch.tensor(coo.sum(axis=0)).reshape(-1,1)/v1
    v3 = torch.tensor(coo.max(axis=0).toarray()).reshape(-1,1)
    v4 = torch.tensor(coo.min(axis=0).toarray()).reshape(-1,1)
    v5 = b_vars.clone().reshape(-1,1)
    v_nodes = torch.cat((v0,v1,v2,v3,v4,v5),1).float()
    c0 = torch.tensor(coo.getnnz(axis=1)).reshape(-1,1)
    c1 = torch.tensor(coo.sum(axis=1)).reshape(-1,1)/c0
    c2 = torch.tensor(m.getAttr("RHS", constrs)).reshape(-1,1)
    c3 = torch.tensor([2 if i=='<' else 1 for i in m.getAttr("Sense", constrs)]).reshape(-1,1)
    c_nodes = torch.cat((c0,c1,c2,c3),1).float()

    tot=int(cols/3)
    col_slides=collections.defaultdict(int)
    for v in variables:
        col_slides[v]=int(tot)
    row_slides=[]
    graph = HeteroData()

    row_slides = collections.defaultdict(int)
    cons = m.getConstrs()
    t=0
    coc=0
    for c in cons:
        name=constraints[t]
        while name not in c.ConstrName:
            t+=1
            name=constraints[t]
        if t>=len(constraints):
            break
        row_slides[name]+=1
    lastv=0
    for v in variables:
        graph[v].x=v_nodes[lastv:lastv+col_slides[v]]
        lastv+=col_slides[v]
    lastc=0
    for c in constraints:
        graph[c].x=c_nodes[lastc:lastc+row_slides[c]]
        lastc+=row_slides[c]
    lastr=0
    for c in constraints:
        lastc=0
        for v in variables:
            mtx = A[lastr:lastr+row_slides[c],lastc:lastc+col_slides[v]].tocoo()
            lastc+=col_slides[v]
            values = mtx.data
            if len(values)==0:
                continue
            row_indices = mtx.row
            col_indices = mtx.col
            indices = np.vstack((row_indices, col_indices))
            graph[c,'c2v',v].edge_index = torch.from_numpy(indices)
            graph[c,'c2v',v].edge_attr = torch.from_numpy(values).reshape(-1,1)
        lastr+=row_slides[c]

    r=m.relax()
    r.setParam('OutputFlag', 0)
    r.optimize()
    #o节点
    graph['obj'].x = torch.tensor([r.objVal]).reshape(-1,1)
    con_indice=[]
    con_attr=[]
    idx=0
    t=0
    for c in r.getConstrs():
        name=constraints[t]
        while name not in c.ConstrName:
            if 'edge_attr' in graph['obj','o2c',name]:
                graph['obj','o2c',name].edge_index = torch.tensor(con_indice).t()
                graph[name,'c2o','obj'].edge_index = torch.tensor(con_indice).t()[[1,0]]
                graph['obj','o2c',name].edge_attr = get_norm(torch.tensor(con_attr).reshape(-1,1))
            t+=1
            name=constraints[t]
            idx=0
            con_indice=[]
            con_attr=[]
        if t>=len(constraints):
            break
        
        if c.Slack < 1e-6:
            con_indice.append((0,idx))
            con_attr.append(c.RHS)
        idx+=1
    graph['obj','o2c',name].edge_index = torch.tensor(con_indice).t()
    graph[name,'c2o','obj'].edge_index = torch.tensor(con_indice).t()[[1,0]]
    graph['obj','o2c',name].edge_attr = get_norm(torch.tensor(con_attr).reshape(-1,1))
    #加入ov边
    for i,v in enumerate(['u','p','s']):
        graph['obj','o2v',v].edge_index = torch.cat((torch.zeros(1,tot, dtype=torch.int64), torch.arange(0,tot).unsqueeze(0)), 0)
        graph[v,'v2o','obj'].edge_index = torch.cat((torch.arange(0, tot).unsqueeze(0), torch.zeros(1, tot, dtype=torch.int64)), 0)
        graph['obj','o2v',v].edge_attr = get_norm(v0[tot*i:tot*(i+1)])

    return graph


In [9]:
def ps_accelerate(sol, data):
    
    Dt = data['Dt']
    # Spin = [a + b for a, b in zip(data['Spin'], data['Dt'])]
    Spin = data['Spin']
    u0 = data['u0']
    p0 = data['p0']
    on_off = data['on_off']

    sol = restore_initial(sol,p0,ramp_down,shut_down,on_off,on,off)
    sol[sol >= 0.9] = 1
    sol = restore_spin_new(sol,Spin,Dt,pmax)
    sol = restore_on_off_new(sol,on_off,on,off)
    sol = restore_ramp(sol,Dt,u0,p0,ramp_down,ramp_up,pmin,pmax,startup,shut_down,on_off,off)
    sol = restore_on_off_new(sol,on_off,on,off)
    sol[sol < 1-(1e-5)] = 0

    return sol

In [14]:
def valid(predict):
    instance_set = ['80_c11_based_8_std']
    opt_loss=[]

    for ins in instance_set:
        json_dir = f'../datasets/{ins}/json/test'
        inf_cnt=0
        files = os.listdir(json_dir)
        files = [file for file in files if file.endswith('.json')][::20]
        for file in files:
            json_file = os.path.join(json_dir, file)
            with open(json_file,'r') as f:
                data = json.load(f)
            Dt = data['Dt']
            Spin = data['Spin']
            u0 = data['u0']
            p0 = data['p0']
            on_off = data['on_off']
            N = len(u0)
            T = len(Dt)
            m1 = uc.get_1bin_model(Dt=Dt,Spin=Spin,ThTime_on_off_init=on_off,Ui0=u0,Pi0=p0)
            
            
            graph=get_Tripartite_graph(m1).to(DEVICE)
            sol = predict(graph).detach().cpu().numpy().reshape(N,T)
            sol = ps_accelerate(sol,data)
            
            u = m1.getVars()[:N*T]
            for i in range(N):
                for t in range(T):
                    m1.addConstr(u[i*T+t]==sol[i,t],name=f'fix_u[{i},{t}]')
            m1.optimize()
            obj=-1
            if m1.status == 2 or m1.status == 9:
                obj = m1.objVal
            if obj == -1:
                inf_cnt+=1
            else:
                opt_loss.append((obj-data['obj'])/data['obj'])
        if len(files)==0:
            print('no file')
    opt_loss=np.array(opt_loss)
    return opt_loss.mean()

In [11]:
optimizer = torch.optim.AdamW(PredictModel.parameters(), lr=LEARNING_RATE,weight_decay=1e-6)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS/10, eta_min=ETA_MIN)
best_val_loss = 0.08
best_optloss=0.01
cur_epoch = 0

In [15]:

for epoch in range(NB_EPOCHS):
    begin=time.time()
    cur_epoch+=1
    train_loss,train_acc = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.6f} Train acc:{train_acc:0.6f}",end='\t')
    valid_loss,valid_acc = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.6f} Valid acc:{valid_acc:0.6f}")
    optloss=valid(PredictModel)
    print("optloss",optloss)
    if valid_loss<best_val_loss:
        cur_epoch=0
        best_val_loss = valid_loss
        print("best_loss",best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    if cur_epoch%20==0:
        torch.save(PredictModel.state_dict(),model_save_path+f'model_{cur_epoch}.pth')
    if optloss<best_optloss:
        best_optloss=optloss
        print("best_optloss",best_optloss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best_acc.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    #100个epochloss不下降就停止
    if cur_epoch>100:
        break
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}  Valid acc:{valid_acc} optloss:{optloss} TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

Epoch 0 Train loss: 0.074486 Train acc:0.972134	Epoch 0 Valid loss: 0.080373 Valid acc:0.969325
optloss 0.04735056918053622
Epoch 1 Train loss: 0.074503 Train acc:0.972109	Epoch 1 Valid loss: 0.079365 Valid acc:0.969475
optloss 0.06894330440851804
best_loss 0.07936501842204148
Epoch 2 Train loss: 0.074405 Train acc:0.972175	Epoch 2 Valid loss: 0.079917 Valid acc:0.969186
optloss 0.08445950567477552
Epoch 3 Train loss: 0.074367 Train acc:0.972197	Epoch 3 Valid loss: 0.079497 Valid acc:0.970044
optloss 0.07812513187981197
Epoch 4 Train loss: 0.074391 Train acc:0.972146	Epoch 4 Valid loss: 0.079964 Valid acc:0.969233
optloss 0.10747884332020942
Epoch 5 Train loss: 0.074337 Train acc:0.972163	Epoch 5 Valid loss: 0.079004 Valid acc:0.969721


GurobiError: License expired 2024-03-14